## Welcome to CS4TB3 Group 14 Final Project Demo

### Via this application, we aim to convert Regular Expressions into Simplified Natural Language

In [ ]:
!pip install -q pydot

In [ ]:
# Setup for Prefix Tree (Trie) generation and graph visualization 
import json
import pydot
from IPython.display import Image, display

def view_pydot(pdot):
    plt = Image(pdot.create_png())
    display(plt)

# Function to return a new dict template
def struct():
    struct = {
        'iw': 'False'
    }
    return struct

# Make Trie out of words
# Sorted by length, then 10 shortest valid regex outputs chosen
def make_trie(words):
    tmp_s = struct()
    root = tmp_s
    words.sort(key=len)
    words = words[0:10]
    for word in words:
        for c in word:
            if c not in tmp_s:
                tmp_s[c] = struct()
            tmp_s = tmp_s[c]
        tmp_s['iw'] = 'True'
        tmp_s = root
        cur_word = []
    return root

# Code borrowed from: https://github.com/ahmednooor/trie_graph with minor modifications
counter = 0
def genTree(words):
    global counter
    rt = {'root': make_trie(words)}
    counter = 0
    def draw(parent_name, child_name):
        global counter
        counter += 1
        p_n = parent_name
        c_n = child_name
        graph.add_node(pydot.Node(p_n, label=parent_name.split('_')[0]))
        graph.add_node(pydot.Node(c_n, label=child_name.split('_')[0]))
        edge = pydot.Edge(p_n, c_n)
        graph.add_edge(edge)

    def visit(node, parent=None):
        global counter
        for k,v in node.items():
            if isinstance(v, dict):
                # We start with the root node whose parent is None
                # we don't want to graph the None node
                k = k + '_' + str(counter)
                if parent:
                    draw(parent, k)
                visit(v, k)
            else:
                # drawing the label using a distinct name
                v = v + '_' + str(counter)
                draw(parent, v)

    graph = pydot.Dot(graph_type='digraph')
    visit(rt)
    plt = Image(graph.create_png())
    display(plt)

In [ ]:
# Setting up API
import requests
baseURL = "https://vxk4urue7yyomdrnme22wjqqku0fgmxc.lambda-url.us-east-1.on.aws/"

# Auxiliary Methods:
def extractPatternData(patterns):
    if(patterns["minimumLength"] == 0):
        print("Empty string is allowed")
    else:
        print(f"Minimum length of generated string is {patterns['minimumLength']}")
    for char in patterns["proceeding"]:
        outStr = ""
        outStr = f"Character '{char}' can only appear after Character '{patterns['proceeding'][char]['prev']}'"
        if(patterns["proceeding"][char]['start']):
            outStr += " or start of the string"
        print(outStr)
    for char in patterns["preceeding"]:
        outStr = ""
        outStr = f"Character '{char}' can only appear before Character '{patterns['preceeding'][char]['next']}'"
        if(patterns["preceeding"][char]['end']):
            outStr += " or end of the string"
        print(outStr)
    for char in patterns["oddEven"]["odd"]:
        print(f"Occurrences of '{char}' is always odd")
    for char in patterns["oddEven"]["even"]:
        print(f"Occurrences of '{char}' is always even")
    if(patterns["popularPatterns"]["emails"]):
        print("Regex is similar to commonly used email validation regex")
    if(patterns["popularPatterns"]["phone"]):
        print("Regex is similar to commonly used phone number validation regex")
        
        
def convertRegexToNL(regexInput = None):
    while(not regexInput):
        print("Please enter a regex to convert:")
        regexInput = input()
    print("Please Wait")
    requestBody = {"input": regexInput}
    x = requests.post(baseURL, data = requestBody)
    if(x.status_code == 200):
        data = x.json()
        print(f"Constuction Breakdown:\n{data['explanation']}")
        print(f"Size of generated sample set: {len(data['permutations'])}")
        print("Patterns:")
        extractPatternData(data["patterns"])
        print("Prefix Tree:")
        genTree(data['permutations'])
    else:
        print("An unexpected error occurred, please try again later")

In [ ]:
def demo():
    print("------------------------\n Select a number below to run the following examples:", flush = True)
    print("""
    1. ab*(a|bc)+
    2. ba(abb|abab|abba|baba)(a)?
    3. Your own (Enter 3, then enter regex when prompted)
          """, flush = True)
    userInput = input()
    while(not userInput):
        print("Please select a number to continue:")
        userInput = input()
    userInput = int(userInput)
    if(userInput == 1):
        convertRegexToNL("ab*(a|bc)+")
    elif(userInput == 2):
        convertRegexToNL("ba(abb|abab|abba|baba)(a)?")
    elif(userInput == 3):
        convertRegexToNL()

In [ ]:
demo()